# E5.1 — Building ion current models from data 

This notebook contains interactive exercises related to the L5 and L6 sessions of the SSCP2018. 

With these exercises we will begin applying some conventional approaches to using voltage-clamp data to parameterize ion channel models. A first step is for us to construct the steady-state activation curve for the fast cardiac sodium current from an existing data set.

Let's import the data.

In [ ]:
import neo
import pylab as plt
import numpy as np
import math
import matplotlib.pylab as plt
%matplotlib inline

In [ ]:
sweep_num = 15 # the number of voltage steps
V_steps = np.linspace(-120, 20, 15) # the voltage step values
SR = 5 #(kHz)
sweep_length = 5000 #(points)
sweep_duration = sweep_length/SR #(ms)
time = np.linspace(0.2, sweep_duration, sweep_length) # construct the time vector
I = np.zeros(shape=(sweep_length,sweep_num)) # initialize the data array

f = neo.io.AxonIO('Active.abf')
bl = f.read_block()
for idx, seg in enumerate(bl.segments):
    data = np.array(seg.analogsignals[0])
    I[:,idx] = data[:,0]
    plt.plot(time, I) 
    plt.xlabel('time [ms]') 
    plt.ylabel('Measured current [pA]'); 
    plt.xlim([260, 280])
    plt.ylim([-3500, 2000])
plt.show()



And with some simple peak-finding we can have the maximum inward $I_{Na}$ at each voltage to construct our steady-state activation curve:



In [ ]:
peak = np.zeros(15) #initialize arrays to store the times, indices, and values of the peak for each voltage step
peak_index = np.zeros(15)
t_peak = np.zeros(15)

# loop to find peaks and store them
for i in range(sweep_num):
    # define a search range for finding the peak inward current
    start = int(5000*(266.6/1000))
    finish = int(1500)
    search = I[start:finish,i] 
    # find the peak value, its index, and the time at the peak
    peak[i] = search.min()
    peak_index[i] = search.argmin()+start
    t_peak[i] = time[int(peak_index[i])]
    # plot a close-up of the time-series for each voltage step and the peak you have picked out 
    plt.plot(time, I,t_peak,peak,'ro'); plt.xlabel('time [ms]'); plt.ylabel('Measured current [pA]');
    plt.xlim([265, 275]); plt.ylim([-3500, 1000])
plt.show()

## Exercise: extracting the steady-state activation curve

Now you have the peak values for the current at each step potential. The next exercises ask you to reduce it to varying degrees of detail, all to get to data that can be used to optimize the activation properties of a Na$^+$ channel model.

#### 1. Plot the steady-state I-V relationship

In [ ]:
plt.figure()

V_steps = np.linspace(-120, 20, 15)
plt.plot(V_steps, peak,V_steps, peak,'ro'); plt.xlabel('Test potential [mV]'); plt.ylabel('Current [pA]')
plt.xlim([-100, 20])
plt.ylim([-3000, 200])
plt.show()

#### 2. Plot the steady-state g-V relationship
Assume an extracellular Na$^+$ concentration of 130 mM and an intracellular concentration of 10 mM 

In [ ]:
# Your code here:

plt.figure()

R = 8.314
T = 300 
z = 1
F = 96485
Co = 130
Ci = 10

E_Na = (R*T/(z*F))*math.log(Co/Ci) # Nernst 
E_Na = E_Na*1000 # mV

EC_force = V_steps-E_Na # Electrochemical force (mV)
g_Na = peak/(EC_force) # conductance (nS)
norm_g_Na = g_Na/g_Na.max() # normalized conductance

plt.plot(V_steps,g_Na,V_steps, g_Na,'go'); plt.xlabel('Test potential [mV]'); plt.ylabel('Conductance [nS]')
plt.xlim([-100, 20])

#### 3. Given the start made below, try to write a routine to calculate and plot the *INactivation* time constants

In [ ]:
from scipy.optimize import curve_fit

tau = np.zeros(15)
# define the decay function to this data range
def exp_decay(t,A,K,C): 
    return np.array(A * np.exp(-K * t) + C)
def exp_fit(t,y): 
    params, cov = curve_fit(exp_decay, t, y)
    A, K, C = params
    return A, K, C

# Your code here:
tau = np.zeros(len(V_steps))
plt.figure()
# loop to find decay constants and store them
for idx, seg in enumerate(bl.segments):
    I = np.array(seg.analogsignals[0])
    I = I[:,0]
    
    # define the range for inactivation fitting
    start = int(5000*(t_peak[idx]/1000))
    finish = int(5000*(330/1000))
    short_I = 0-I[start:finish]
    short_t = time[start:finish]-time[start-1]
    
    # fit the exponential decay function
    A, K, C = exp_fit(short_t,short_I)
    
    # plot the data
    plt.plot(short_t, short_I,'b')
    
    # plot the fits
    fit_I = exp_decay(short_t,A,K,C)
    plt.plot(short_t, fit_I,'r')
    
    # store the time constants
    tau[idx] = 1/K
plt.show()
plt.figure()
plt.plot(V_steps[6:], tau[6:])